**Combined Preprocessing Pipeline**


In [9]:

!pip install opendatasets


In [10]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/zlatan599/garbage-dataset-classification")

Skipping, found downloaded files in "./garbage-dataset-classification" (use force=True to force download)


In [11]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

**Load training and validation data and resize the images**

In [21]:
import tensorflow as tf
from tensorflow.keras.layers import Rescaling

data_dir = "/content/garbage-dataset-classification/Garbage_Dataset_Classification/"

# Create datasets (80/20 split)
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(224, 224),
    batch_size=32
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(224, 224),
    batch_size=32
)

Found 13901 files belonging to 1 classes.
Using 11121 files for training.
Found 13901 files belonging to 1 classes.
Using 2780 files for validation.


**Rescaling**

In [22]:
train_ds = train_ds.map(lambda x, y: (Rescaling(1./255)(x), y))
val_ds   = val_ds.map(lambda x, y: (Rescaling(1./255)(x), y))

**Geometric Augmentation**

In [25]:
import tensorflow as tf
from tensorflow.keras import layers
geometric_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomTranslation(0.1, 0.1),
])

def apply_geometric_augmentation(image, label):
    image = geometric_augmentation(image, training=True)
    return image, label

train_ds = train_ds.map(apply_geometric_augmentation)


**Color_Augmentation**

In [26]:
color_augmentation = tf.keras.Sequential([
    layers.RandomBrightness(0.2),
    layers.RandomContrast(0.2),
    layers.RandomSaturation(0.2),
    layers.RandomHue(0.1),
])

def apply_color_augmentation(image, label):
    image = color_augmentation(image, training=True)
    return image, label

train_ds = train_ds.map(apply_color_augmentation)

**Gaussian Noise**

In [27]:
from tensorflow.keras.layers import GaussianNoise

def add_gaussian_noise(image, stddev=0.05):
    noise = tf.random.normal(shape=tf.shape(image), mean=0.0, stddev=stddev)
    return tf.clip_by_value(image + noise, 0.0, 1.0)

def apply_noise_augmentation(image, label):
    image = add_gaussian_noise(image)
    return image,label

train_ds = train_ds.map(apply_noise_augmentation)


**Histogram Equalization**

In [28]:
def histogram_equalization_rgb(image):

    # Convert to YUV color space
    image_yuv = tf.image.rgb_to_yuv(image)

    # Apply histogram equalization to Y channel (luminance)
    y_channel = image_yuv[..., 0:1]

    # Enhanced contrast adjustment (simulating histogram equalization)
    y_eq = tf.image.adjust_contrast(y_channel, 2.0)
    y_eq = tf.clip_by_value(y_eq, 0.0, 1.0)

    # Combine back with original UV channels
    image_yuv_eq = tf.concat([y_eq, image_yuv[..., 1:2], image_yuv[..., 2:3]], axis=-1)

    # Convert back to RGB
    return tf.image.yuv_to_rgb(image_yuv_eq)

def apply_histogram_equalization(image, label):
    image = histogram_equalization_rgb(image)
    return image, label

train_ds = train_ds.map(apply_histogram_equalization)
